In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K 
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import regularizers
import tensorflow_model_optimization as tfmot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [105]:
data = pd.read_pickle('./data/images_df_numerical.pkl')
classes = data["Species"].unique()
number_of_classes = classes.size
X, y = data['data'], data['Species']
# Wir hatten massive Probleme mit der Begrenztheit unseres RAMs, weshalb wir versucht haben die Usage an mehreren Stellen zu reduzieren
X, y = np.stack(X).astype(np.uint8), y.to_numpy().astype(np.uint8)


In [106]:
image_size = X[0].size
samples = X.size
with open("./data/meta.json","r") as file:  # Shape der Bilder nach dem resizen aus dem data_prep notebook
    image_meta = json.load(file) 
image_shape = (image_meta['h'],image_meta['w'],image_meta['c'])

In [107]:
X = X.reshape((-1,) + image_shape)
print(f"Image has shape: {image_shape}")

Image has shape: (82, 128, 3)


In [108]:
kfold = StratifiedKFold(n_splits=2, shuffle=True)

In [109]:
def train_val_test_split(train_indezes, test_indezes):
    X_train, X_val, y_train, y_val = train_test_split(X[train_indezes], y[train_indezes], test_size=0.2,stratify=y[train_indezes], random_state=42)
    X_test, y_test = X[test_indezes], y[test_indezes]
    
    # Das speichern als Tensor spart RAM
    return (tf.convert_to_tensor(X_train),
            tf.convert_to_tensor(y_train), 
            tf.convert_to_tensor(X_val),
            tf.convert_to_tensor(y_val),
            tf.convert_to_tensor(X_test),
            tf.convert_to_tensor(y_test))

In [110]:
print(image_shape)

(82, 128, 3)


In [111]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.001, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32

filter_sizes = [32,32,32,32,32,1]

def create_autoencoder_model(silent=True):
    # for architecture, see: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8456308
    #encoder
    input_en = tf.keras.Input(shape=image_shape, name="EN-IN")
    # downsampling unit 1
    x = tf.keras.layers.Conv2D(filter_sizes[0], (3, 3), activation='PReLU', padding='same', strides=1, name="EN-DS1-1")(input_en)
    x = tf.keras.layers.Conv2D(filter_sizes[1], (3, 3), activation='PReLU', padding='same', strides=2, name="EN-DS1-2")(x)
    # downsampling unit 2
    x = tf.keras.layers.Conv2D(filter_sizes[2], (3, 3), activation='PReLU', padding='same', strides=1, name="EN-DS2-1")(x)
    x = tf.keras.layers.Conv2D(filter_sizes[3], (3, 3), activation='PReLU', padding='same', strides=2, name="EN-DS2-2")(x)
    # downsampling unit 3
    x = tf.keras.layers.Conv2D(filter_sizes[4], (3, 3), activation='PReLU', padding='same', strides=1, name="EN-DS3-1")(x)
    output_en = tf.keras.layers.Conv2D(filter_sizes[5], (3, 3), activation='PReLU', padding='same', strides=2, name="EN-DS3-2")(x)

    encoder = tf.keras.Model(input_en, output_en, name="Encoder")
    if not silent:
        encoder.summary()
    
    # decoder
    input_de = tf.keras.Input(shape=(11,16,filter_sizes[5]), name="DE-IN")
    # upsampling unit 1
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[5], (3, 3), activation='PReLU', padding='same', strides=1, name="DE-DS1-1")(input_de)
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[4], (4, 4), activation='PReLU', padding='same', strides=2, name="DE-DS1-2")(x)
    # upsampling unit 2
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[3], (3, 3), activation='PReLU', padding='same', strides=1, name="DE-DS2-1")(x)
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[2], (4, 4), activation='PReLU', padding='same', strides=2, name="DE-DS2-2")(x)
    # upsampling unit 3
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[1], (3, 3), activation='PReLU', padding='same', strides=1, name="DE-DS3-1")(x)
    x = tf.keras.layers.Conv2DTranspose(filter_sizes[0], (4, 4), activation='PReLU', padding='same', strides=2, name="DE-DS3-2")(x)
    # crop to match shape
    x = tf.keras.layers.Cropping2D(cropping=((4,2),(0,0)))(x)
    decoded = tf.keras.layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

    
    decoder = tf.keras.Model(input_de, decoded, name="Decoder")
    if not silent:
        decoder.summary()

    #autoencoder
    input_ae = tf.keras.Input(shape=image_shape)
    x = encoder(input_ae)
    output_ae = decoder(x)
    
    autoencoder = tf.keras.Model(input_ae, output_ae, name="Autoencoder")
    if not silent:
        autoencoder.summary()

    return autoencoder

In [112]:
# Stoppt wenn bei 30 Epochen die Validation-Accuracy nicht um mehr als 0.001 besser wird
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.01, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32
dropout_rate = 0.4
weight_decay_alpha = 0.01

def create_cnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=image_shape, name="aaa"))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))

    return model

In [113]:
def create_combined_model(autoencoder_model, cnn_model):
    # Eingabe für beide Modelle
    input_combined = tf.keras.layers.Input(shape=image_shape, name="combined_input")

    # CNN-Modell aufrufen und Ausgabe erhalten
    cnn_output = cnn_model(input_combined)

    # Autoencoder-Modell aufrufen und Ausgabe erhalten
    autoencoder_output = autoencoder_model(input_combined)

    # Gemeinsames Modell erstellen
    combined_model = tf.keras.Model(inputs=input_combined, outputs=[autoencoder_output, cnn_output], name="combined_model")

    return combined_model

In [114]:
def fit_model(model, X_train, y_train, X_val=None, y_val=None):
    # Das Model macht selbst den Validation Split
    if X_val is None or y_val is None:
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
            validation_split=0.2,
            verbose=1)
    # Wir übergeben dem Model Validation Daten
    elif X_val is not None and y_val is not None:
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
            validation_data=(X_val,y_val),
            verbose=1)
    return history

In [115]:
cnn_model = create_cnn_model()
autoencoder_model = create_autoencoder_model(False)
combined_model = create_combined_model(autoencoder_model, cnn_model)

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EN-IN (InputLayer)          [(None, 82, 128, 3)]      0         
                                                                 
 EN-DS1-1 (Conv2D)           (None, 82, 128, 32)       336768    
                                                                 
 EN-DS1-2 (Conv2D)           (None, 41, 64, 32)        93216     
                                                                 
 EN-DS2-1 (Conv2D)           (None, 41, 64, 32)        93216     
                                                                 
 EN-DS2-2 (Conv2D)           (None, 21, 32, 32)        30752     
                                                                 
 EN-DS3-1 (Conv2D)           (None, 21, 32, 32)        30752     
                                                                 
 EN-DS3-2 (Conv2D)           (None, 11, 16, 1)         465 

Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 DE-IN (InputLayer)          [(None, 11, 16, 1)]       0         
                                                                 
 DE-DS1-1 (Conv2DTranspose)  (None, 11, 16, 1)         186       
                                                                 
 DE-DS1-2 (Conv2DTranspose)  (None, 22, 32, 32)        23072     
                                                                 
 DE-DS2-1 (Conv2DTranspose)  (None, 22, 32, 32)        31776     
                                                                 
 DE-DS2-2 (Conv2DTranspose)  (None, 44, 64, 32)        106528    
                                                                 
 DE-DS3-1 (Conv2DTranspose)  (None, 44, 64, 32)        99360     
                                                                 
 DE-DS3-2 (Conv2DTranspose)  (None, 88, 128, 32)       3768

In [116]:
# Abspeichern der Erebnisse jedes Splits für die Confusion-Matrix
true_labels = list()
pred_labels = list()
train_accuracies = list()
test_accuracies = list()
val_accuracies = list()
train_losses = list()
val_losses = list()
test_losses = list()



for train_indezes, test_indezes in kfold.split(X, y):
    # wir löschen das Model der letzten Iteration aus dem Cache um RAM zu sparen
    K.clear_session()

    # Daten splitten
    X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(train_indezes, test_indezes)

    # Resample nur Trainings- und Validationmenge
    #X_train, y_train = resample_after_split(X_train, y_train)
    #X_val, y_val = resample_after_split(X_val, y_val)

    # Das Model wird hier compiled, damit wir nach jeder Iteration das alte Model aus dem Cache löschen können  Adam(0.001)
    combined_model.compile(optimizer=Adam(learning_rate=0.001), loss=['mse', 'sparse_categorical_crossentropy'], metrics=['mse', 'accuracy'])
    history = combined_model.fit(X_train, [X_train, y_train], epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_val, [X_val, y_val]), callbacks=[early_stopping])
    
    #history = fit_model(combined_model, X_train, y_train, X_val, y_val)

    # Für die Confusion Matrix
    predictions = np.argmax(combined_model.predict(X_test), axis=-1)
    true_labels.extend(y_test)
    pred_labels.extend(predictions)

    # Für die Accuracy-Curves
    train_accuracies.extend(history.history['accuracy'])
    val_accuracies.extend(history.history['val_accuracy'])

    # Für die Loss-Curves
    train_losses.extend(history.history['loss'])
    val_losses.extend(history.history['val_loss'])

    loss, accuracy = combined_model.evaluate(X_test, y_test)
    test_accuracies.append(accuracy)
    test_losses.append(loss)

Epoch 1/200


/home/marcel/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


47/47 [==============================] - 55s 1s/step - loss: 4.3540 - Autoencoder_loss: 0.2444 - sequential_loss: 1.5931 - Autoencoder_mse: 0.2444 - Autoencoder_accuracy: 1.0000 - sequential_mse: 2.3994 - sequential_accuracy: 0.5652 - val_loss: 2.1968 - val_Autoencoder_loss: 0.2384 - val_sequential_loss: 1.3690 - val_Autoencoder_mse: 0.2384 - val_Autoencoder_accuracy: 1.0000 - val_sequential_mse: 2.3958 - val_sequential_accuracy: 0.5657
Epoch 2/200
47/47 [==============================] - 51s 1s/step - loss: 1.8153 - Autoencoder_loss: 0.2330 - sequential_loss: 1.2886 - Autoencoder_mse: 0.2330 - Autoencoder_accuracy: 1.0000 - sequential_mse: 2.4225 - sequential_accuracy: 0.5652 - val_loss: 1.5867 - val_Autoencoder_loss: 0.2273 - val_sequential_loss: 1.2474 - val_Autoencoder_mse: 0.2273 - val_Autoencoder_accuracy: 1.0000 - val_sequential_mse: 2.4203 - val_sequential_accuracy: 0.5657
Epoch 3/200
47/47 [==============================] - 50s 1s/step - loss: 1.5291 - Autoencoder_loss: 0.2220

KeyboardInterrupt: 

In [ ]:
confusion_matrix = confusion_matrix(true_labels, pred_labels)
fig, ax = plt.subplots(figsize=(8, 6))
cm_display = ConfusionMatrixDisplay(confusion_matrix, display_labels=classes)
cm_display.plot(ax=ax, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
epochs = len(train_losses)
plt.figure(figsize=(12, 4))

# Plotting Loss curves
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting Accuracy curves
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig("./cnn_files/loss_and_accuraccy.png")
plt.show()

In [ ]:
print("Unser Modell:")
print(f"Avg. Val Accuracy: {sum(val_accuracies) / len(val_accuracies)}")
print(f"Best Val Accuracy: {max(val_accuracies)}")
print(f"Avg. Test Accuracy: {sum(test_accuracies) / len(test_accuracies)}")
print(f"Best Test Accuracy: {max(test_accuracies)}")
print(f"Avg. Test Loss: {sum(test_losses) / len(test_losses)}")